# Simulation template.

In this notebook we produce the results of the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 635 ms, sys: 56.3 ms, total: 692 ms
Wall time: 690 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = "/home/linneu/ssh_csvs/ssh1/periodic_100_6561.csv"   
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = "/home/linneu/ssh_simulations/ssh1/periodic_100_6561"
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 6.68 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy"}
allowed_windings = [0, 1]
val_split = 0.9
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/linneu/ssh_csvs/ssh1/periodic_220_6561.csv"
model_name = "DecisionTreeClassifier"
simulation_dir = "/home/linneu/ssh_simulations/ssh1/periodic_220_6561"
random_state = 345


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/linneu/anaconda3/envs/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1443420 entries, 0 to 1443419
Columns: 228 entries, id to feat219
dtypes: float64(223), int32(3), object(2)
memory usage: 2.4+ GB
CPU times: user 43.7 s, sys: 2.24 s, total: 45.9 s
Wall time: 46.7 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.067420,0.067420,...,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.095346,0.095307,...,0.091484,0.090680,0.092868,0.092214,0.093950,0.093447,0.094725,0.094376,0.095191,0.094997
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.095346,0.095191,...,0.080210,0.077137,0.085564,0.083022,0.089801,0.087826,0.092868,0.091484,0.094725,0.093950
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.067420,-0.067420,...,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.095346,-0.095307,...,0.091484,-0.090680,0.092868,-0.092214,0.093950,-0.093447,0.094725,-0.094376,0.095191,-0.094997


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  220
n_hamiltonians:  6561
n_ts:  2
CPU times: user 377 µs, sys: 20 µs, total: 397 µs
Wall time: 244 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.9358329522938577
% val:  0.0
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  6140
number of val hamiltonians:  0
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [01:01<1:41:38, 61.60s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [02:03<1:40:30, 61.54s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [1:42:13<00:00, 61.73s/it]

running experiments: 100%|██████████| 100/100 [1:42:13<00:00, 61.33s/it]

CPU times: user 1h 41min 26s, sys: 50.7 s, total: 1h 42min 17s
Wall time: 1h 42min 13s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.067420,0.067420,...,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.095346,0.095307,...,0.091484,0.090680,0.092868,0.092214,0.093950,0.093447,0.094725,0.094376,0.095191,0.094997
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.095346,0.095191,...,0.080210,0.077137,0.085564,0.083022,0.089801,0.087826,0.092868,0.091484,0.094725,0.093950
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.067420,-0.067420,...,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.095346,-0.095307,...,0.091484,-0.090680,0.092868,-0.092214,0.093950,-0.093447,0.094725,-0.094376,0.095191,-0.094997
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.095346,-0.095191,...,0.080210,-0.077137,0.085564,-0.083022,0.089801,-0.087826,0.092868,-0.091484,0.094725,-0.093950
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.095346,0.085564,...,0.013569,0.029464,0.083022,-0.053818,0.087826,-0.095191,0.024239,-0.062439,-0.058223,0.018935
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.000142,-0.002581,...,0.026726,-0.029329,0.021458,-0.024102,0.016119,-0.018796,0.010728,-0.013429,0.005302,-0.008018
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.000068,-0.005375,...,0.051491,-0.055988,0.042008,-0.046825,0.031977,-0.037053,0.021529,-0.026797,0.010801,-0.016191
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.095346,0.094725,...,-0.039608,0.029463,-0.058223,0.049236,-0.073811,0.066450,-0.085563,0.080210,-0.092868,0.089801


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 386 ms, sys: 16 µs, total: 386 ms
Wall time: 384 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
1004293,4564,H_0.8_-0.6_0.000464825.dat,0.80,-0.60,0.000465,0,0,train,0.016788,-0.035847,...,0.092349,0.065476,0.084577,0.048076,0.072409,0.028177,0.056477,0.006814,0.037610,-0.014904
1388580,6311,H_1.85_1.7_-0.0122955.dat,1.85,1.70,-0.012296,0,0,test,0.095206,-0.034981,...,0.089931,-0.008577,-0.058541,-0.044120,0.008566,0.082809,0.044130,-0.095206,-0.082814,0.077377
279950,1272,H_-1.25_0.85_0.000439002.dat,-1.25,0.85,0.000439,0,0,val,-0.000308,-0.051151,...,-0.086601,0.094442,-0.032331,-0.021136,0.092135,-0.090824,0.016563,0.036681,-0.094970,0.084546
201574,916,H_-1.45_-0.75_-0.00116207.dat,-1.45,-0.75,-0.001162,0,0,val,0.095346,-0.093588,...,-0.091481,0.094932,-0.062431,0.075061,-0.013559,0.031359,0.039617,-0.022299,0.080216,-0.068878
684425,3111,H_-0.1_-0.35_1.00152.dat,-0.10,-0.35,1.001518,1,1,val,-0.000743,0.009203,...,-0.052172,0.059050,-0.042735,0.050131,-0.032740,0.040559,-0.022319,0.030459,-0.011607,0.019961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485828,2208,H_-0.65_-0.95_1.00343.dat,-0.65,-0.95,1.003435,1,1,train,0.095340,-0.077000,...,0.040576,-0.083136,-0.052934,-0.002955,-0.095246,0.080085,-0.045435,0.085666,0.048320,0.008391
1351849,6144,H_1.75_1.45_-0.00524251.dat,1.75,1.45,-0.005243,0,0,val,0.001064,-0.078664,...,0.072750,-0.092233,-0.092479,0.072107,0.093230,-0.037184,-0.074850,-0.005369,0.041111,0.046820
613125,2786,H_-0.3_-0.4_1.00434.dat,-0.30,-0.40,1.004338,1,1,train,0.000665,0.092158,...,-0.087004,-0.060525,0.078329,-0.031835,0.006107,0.093403,-0.084636,-0.064632,0.081304,-0.026652
1051286,4778,H_0.9_2_1.00197.dat,0.90,2.00,1.001972,1,1,val,0.000256,-0.053577,...,0.094412,0.070442,-0.093396,-0.087939,0.081517,0.095207,-0.060157,-0.091402,0.031800,0.076965


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.09358329522938577
% val:  0.8422496570644719
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  614
number of val hamiltonians:  5526
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [5244, 4886, 4568, 5091, 416, 4011, 3712, 270, 550, 2711, 4029, 2688, 4661, 4081, 5461, 1704, 4979, 5438, 5203, 3229, 5042, 4631, 2313, 1447, 2208, 4688, 2639, 6145, 3324, 2535, 414, 4178, 3949, 1256, 184, 1298, 962, 3818, 1948, 3427, 603, 5614, 5948, 3745, 2427, 3028, 3561, 4949, 4116, 6335, 2330, 910, 5966, 2043, 1858, 1611, 697, 384, 6208, 539, 5477, 4944, 1812, 4431, 3584, 6, 2796, 4596, 5306, 1905, 672, 5744, 263, 4671, 6367, 3297, 2420, 6020, 3296, 366, 24, 5753, 6262, 3246, 2463, 5516, 729, 3474, 438, 1381, 1439, 3481, 4459, 231, 1015, 3618, 3518, 1575, 2491, 4109, 5781, 3672, 513, 2559, 6462, 3668, 2236, 5125, 991, 556, 581, 5533, 430, 552, 6461, 3456, 1018, 30, 2507, 4968, 2385, 4617, 4057, 613, 1468, 584, 154, 39

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 239 µs, sys: 2 µs, total: 241 µs
Wall time: 155 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.472727,0.527273,999,1
1,-2.0,-1.95,test,0.686364,0.313636,0,0
2,-2.0,-1.90,test,0.754545,0.245455,0,0
3,-2.0,-1.85,test,0.790909,0.209091,0,0
4,-2.0,-1.80,val,0.918182,0.081818,0,0
...,...,...,...,...,...,...,...
6556,2.0,1.80,val,0.904545,0.095455,0,0
6557,2.0,1.85,test,0.800000,0.200000,0,0
6558,2.0,1.90,test,0.800000,0.200000,0,0


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1443420


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,1,test
2,0,999,0,test
3,0,999,0,test
4,0,999,1,test
...,...,...,...,...
1443415,6560,999,1,test
1443416,6560,999,1,test
1443417,6560,999,0,test
1443418,6560,999,1,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9914643174415161,
 'eigenvector_val': 0.9746051722436088,
 'eigenvector_test': 0.7856118881118881,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 1.0,
 'hamiltonian_test': 1.0}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}